In [1]:
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime

from typing import Callable, List

from psycopg import sql

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch
from dmp.postgres_interface.element.column import Column
from dmp.postgres_interface.element.table import Table
from dmp.postgres_interface.element.column_group import ColumnGroup

from dmp.util.butter_e_export import *

pd.options.display.max_seq_items = None

%load_ext autoreload
%autoreload 2
credentials = jobqueue.load_credentials("dmp")


2023-11-17 14:17:06.968352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TODO: convert this into two steps:
    1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

          SELECT
                  {run}.{run_id}
          FROM
                  {run},
                  {job_status},
                  {job_data}
          WHERE TRUE
                  AND {run}.batch like {pattern}
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
              AND {job_status}.status = 2
          ORDER BY experiment_id, run_id;

    2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
      -> partition by the attributes we care about querying by (dataset, size, shape, depth)

              SELECT
                  {columns}
              FROM
                  {run},
                  {job_status},
                  {job_data}
              WHERE TRUE
                  AND {job_status}.id = {run}.run_id
                  AND {job_status}.id = {job_data}.id
                  AND {job_status}.status = 2
                  AND {run}.{run_id} IN ({ids})

        pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
        results = pool.uimap(download_chunk, chunks)
        for num_rows, chunk in results:
            num_stored += 1
            print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

    extra credit) extract butter data matching this as well into a new dataset
    extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
    

In [2]:
from psycopg import ClientCursor


print(f"run vars {vars(run)}")

columns = (
    run
    + ColumnGroup(*[c for c in job_status.columns if c.name != "id"])
    + job_data.command
)
print(columns.names)


def passthrough(row, index, value, column, data):
    data[column.name] = value


column_converters: List[Callable] = [passthrough for _ in columns]


def flatten_json(json_obj, destination=None, parent_key="", separator="_"):
    if isinstance(destination, dict):
        flattened = destination
    else:
        flattened = {}

    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


column_converters[
    columns.get_index_of(job_data.command)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)
column_converters[
    columns.get_index_of(run.run_data)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)


def parquet_to_dataframe(row, index, value, column, data):
    with io.BytesIO(value) as buffer:
        data[column.name] = (
            pyarrow.parquet.read_table(pyarrow.PythonFile(buffer, mode="r"))
            .to_pandas()
            .sort_values(by="epoch")
        )


column_converters[columns.get_index_of(run.run_history)] = parquet_to_dataframe
column_converters[columns.get_index_of(run.run_extended_history)] = parquet_to_dataframe


dfs = []

run vars {'_name': 'run', '_columns': (), '_index': None}
('experiment_id', 'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id', 'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory', 'host_name', 'batch', 'run_data', 'run_history', 'run_extended_history', 'queue', 'status', 'priority', 'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent', 'command')
{Column(_name='experiment_id', type_name='uuid'): 0, Column(_name='run_timestamp', type_name='timestamp'): 1, Column(_name='run_id', type_name='uuid'): 2, Column(_name='job_id', type_name='uuid'): 3, Column(_name='seed', type_name='bigint'): 4, Column(_name='slurm_job_id', type_name='bigint'): 5, Column(_name='task_version', type_name='smallint'): 6, Column(_name='num_nodes', type_name='smallint'): 7, Column(_name='num_cpus', type_name='smallint'): 8, Column(_name='num_gpus', type_name='smallint'): 9, Column(_name='gpu_memory', type_name='integer'): 10, Column(_name='host_name', type_name='text'): 11,

In [13]:
def get_ids():
    with ConnectionManager(credentials) as connection:
        """
        TODO: convert this into two steps:
        1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

            SELECT
                    {run}.{run_id}
            FROM
                    {run},
                    {job_status},
                    {job_data}
            WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                AND {job_status}.status = 2
            ORDER BY experiment_id, run_id;

        2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
        -> partition by the attributes we care about querying by (dataset, size, shape, depth)

                SELECT
                    {columns}
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {run}.{run_id} IN ({ids})

            pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
            results = pool.uimap(download_chunk, chunks)
            for num_rows, chunk in results:
                num_stored += 1
                print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

        extra credit) extract butter data matching this as well into a new dataset
        extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
        """
        query = psycopg.sql.SQL(
            """
                SELECT
                    {run}.run_id
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {job_data}.command @> {json_data}::jsonb
                ORDER BY experiment_id, run_id
                LIMIT 10;
            """ 
        ).format(
            run=run.identifier,
            job_status=job_status.identifier,
            job_data=job_data.identifier,
            pattern=sql.Literal("%energy%"),
            json_data=sql.Literal('''{
            "model": {
                "shape": "rectangle",
                "depth": 5
            },
            "dataset": {
                "name":"sleep"
            }
        }'''

            )
        )

        with ClientCursor(connection) as c:
            print(c.mogrify(query))

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            ids = cursor.fetchall()
            rows = []
            for id in ids:
                rows.append(str(id[0]))
            return rows
        
rows = get_ids()
print(rows)


                SELECT
                    "run".run_id
                FROM
                    "run",
                    "job_status",
                    "job_data"
                WHERE TRUE
                    AND "run".batch like '%energy%'
                    AND "job_status".id = "run".run_id
                    AND "job_status".id = "job_data".id
                    AND "job_status".status = 2
                    AND "job_data".command @> '{
            "model": {
                "shape": "rectangle",
                "depth": 5
            },
            "dataset": {
                "name":"sleep"
            }
        }'::jsonb
                ORDER BY experiment_id, run_id
                LIMIT 10;
            
['50c8b611-a33e-40c8-9756-5a86da78235b', '8472b423-9e1a-4d1a-9d87-14d6154c3016', '6da9a6ae-ea1d-418e-bdac-4a196fe3765b', 'a56ccce5-5090-469d-b47f-cce5dc8003ec', '787b0956-a913-42cf-b1c9-814a2e383233', 'ca64ea3e-4b8a-45b8-b8c0-c5ba97f919a7', '02cf0870-c811-46a0-a9f0-

In [12]:
rows

['caa96421-58f6-435e-afa3-7bcf26b8af56',
 '9329da14-34c9-45ec-8e85-a59c4403767a',
 'ce53fcdf-2220-47b7-9b88-67b137a3775a',
 '0b1ae00c-8891-429f-bd78-3ec403102d95',
 'df1e243e-7296-4c05-b2b1-b9240f90c34c',
 '0ff5a77f-5154-42ba-891f-5764f95a5879',
 '44a97541-44fe-477f-bf3c-43b8cccd33fa',
 '07700714-85d3-4a1a-a0c3-3070c91cb0e4',
 '7bb1f18f-da22-4370-8cfe-751c83113822',
 'c411140c-ae39-413c-88da-742cac2714cb']

In [24]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
def get_data(run_id):
    print(run_id)
    with ConnectionManager(credentials) as connection:
        """
        TODO: convert this into two steps:
        1) Get the ids of the runs we want to extract (in energy batches) ordered by experiment_id

            SELECT
                    {run}.{run_id}
            FROM
                    {run},
                    {job_status},
                    {job_data}
            WHERE TRUE
                    AND {run}.batch like {pattern}
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                AND {job_status}.status = 2
            ORDER BY experiment_id, run_id;

        2) In parallel (using the multiprocessing lib), extract blocks of ids into a partitioned parquet file
        -> partition by the attributes we care about querying by (dataset, size, shape, depth)

                SELECT
                    {columns}
                FROM
                    {run},
                    {job_status},
                    {job_data}
                WHERE TRUE
                    AND {job_status}.id = {run}.run_id
                    AND {job_status}.id = {job_data}.id
                    AND {job_status}.status = 2
                    AND {run}.{run_id} IN ({ids})

            pool = multiprocessing.ProcessPool(multiprocessing.cpu_count())
            results = pool.uimap(download_chunk, chunks)
            for num_rows, chunk in results:
                num_stored += 1
                print(f"Stored {num_rows} in chunk {chunk}, {num_stored} / {len(chunks)}.")

        extra credit) extract butter data matching this as well into a new dataset
        extra credit) make a summary dataset that summarizes the quartiles of # epochs to reach target test loss levels
        """
        query = psycopg.sql.SQL(
            """
                SELECT
                    {columns}
                FROM
                    {run}, 
                    {job_status},
                    {job_data}
                WHERE run_id = {run_id};
            """ 
        ).format(
            columns=columns.columns_sql,
            run_id=sql.Literal(run_id),
            run=run.identifier,
            job_data=job_data.identifier,
            job_status=job_status.identifier,
        )

        with ClientCursor(connection) as c:
            print(c.mogrify(query))

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)
            rows = cursor.fetchall()
            print(rows)

get_data(rows[0])

50c8b611-a33e-40c8-9756-5a86da78235b

                SELECT
                    "experiment_id","run_timestamp","run_id","job_id","seed","slurm_job_id","task_version","num_nodes","num_cpus","num_gpus","gpu_memory","host_name","batch","run_data","run_history","run_extended_history","queue","status","priority","start_time","update_time","worker","error_count","error","parent","command"
                FROM
                    "run", 
                    "job_status",
                    "job_data"
                WHERE run_id = '50c8b611-a33e-40c8-9756-5a86da78235b';
            


KeyboardInterrupt: 

In [ ]:
data

In [18]:
path = "./batch=energy_1/num_nodes=1/num_gpus=2/num_cpus=2/56e4c12eae784e308d8422920a916930-0.parquet"

In [21]:
# pyarrow read parquet from path as table
table = pq.read_table(path)
pd = table.to_pandas()

In [22]:
pd

,experiment_id,run_timestamp,run_id,job_id,seed,slurm_job_id,task_version,gpu_memory,host_name
0,44349db1-47c2-1f11-8011-14328aef8e23,2023-02-09 05:56:23.541736+00:00,a56ccce5-5090-469d-b47f-cce5dc8003ec,a56ccce5-5090-469d-b47f-cce5dc8003ec,1663278620,10373625,3,13312,r104u37
